This notebook is only used for development.

## TODO

- [x] try masking and erode to remove bands
- [x] write output as a COG
- [x] crop to AOI

In [32]:
%load_ext autoreload
%autoreload 2
# import pyroSAR
from pyroSAR.snap.auxil import Workflow, gpt, groupbyWorkers
from pyroSAR import identify_many, identify
from eo_tools.auxils import get_burst_geometry
import geopandas as gpd
import rasterio as rio
import numpy as np
from rasterio import merge, mask
from rasterio.io import MemoryFile
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
from scipy.ndimage import binary_erosion
from glob import glob


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
infiles = ['/data/S1/S1A_IW_SLC__1SDV_20230826T180527_20230826T180554_050050_0605B2_E768.zip',
           '/data/S1/S1A_IW_SLC__1SDV_20230814T180526_20230814T180553_049875_05FFB4_7D92.zip']
outdir = '/data/res/'
aoi_path = '/data/aoi_bretagne.geojson'
# graph_path = '../graph/TOPSAR Coreg Interferogram IW2 to GeoTiff.xml'
# graph_path = '../graph/TOPSAR Coreg Coherence VV IW2 to GeoTiff.xml'
graph_path = '../graph/TOPSAR_coh_geocode_IW_to_geotiff.xml'

In [3]:
# let's find what iw and bursts intersect AOI
df_geo = get_burst_geometry(infiles[0], 
                   target_subswaths=['IW1', 'IW2','IW3'], 
                   polarization='VV')
aoi = gpd.read_file(aoi_path)

In [53]:
ID = identify('/data/tmp/IW2_VV_26Aug2023_14Aug2023_slice_9_coreg.dim')
print(ID)

pyroSAR ID object of type BEAM_DIMAP
acquisition_mode: IW
cycleNumber: 300
frameNumber: 394674
lines: 4192
orbit: A
orbitNumber_abs: 50050
orbitNumber_rel: 103
polarizations: ['VV']
product: SLC
projection: +proj=longlat +datum=WGS84 +no_defs
samples: 26057
sensor: S1A
spacing: (2.329562, 13.90313)
start: 20230826T180537
stop: 20230826T180546


In [46]:
from pathlib import Path
paths = Path('/data/tmp/IW3_VV_26Aug2023_14Aug2023_slice_9_coreg.data/').glob('*.img')
imgnames = set([p.stem[2:] for p in paths])
for name in imgnames:
    print(name)

IW3_VV_mst_26Aug2023
IW3_VV_slv1_14Aug2023


In [ ]:
df_sel = df_geo[df_geo.intersects(aoi.geometry[0])]
unique_subswaths = df_sel['subswath'].unique()

In [ ]:
wfl = Workflow(graph_path)
wfl['Read'].parameters['file'] = infiles[0]
wfl['Read(2)'].parameters['file'] = infiles[1]
for subswath in unique_subswaths[:]:
    print(f"Subswath {subswath}, bursts {df_sel[df_sel['subswath']==subswath]['burst'].values}")
    wfl['TOPSAR-Split'].parameters['subswath'] = subswath
    wfl['TOPSAR-Split(2)'].parameters['subswath'] = subswath
    bursts = df_sel[df_sel['subswath']==subswath]['burst'].values
    burst_min = bursts.min()
    burst_max = bursts.max()
    wfl['TOPSAR-Split'].parameters['firstBurstIndex'] = burst_min
    wfl['TOPSAR-Split'].parameters['lastBurstIndex'] = burst_max
    wfl['TOPSAR-Split(2)'].parameters['firstBurstIndex'] = burst_min
    wfl['TOPSAR-Split(2)'].parameters['lastBurstIndex'] = burst_max
    wfl['Write'].parameters['file'] = f"{outdir}/{subswath}_COH_geo.tif"
    wfl.write('/tmp/graph.xml')
    grp = groupbyWorkers('/tmp/graph.xml', n=1)
    gpt('/tmp/graph.xml', groups=grp, tmpdir='/data/tmp/')

    # remove dark edges
    with rio.open(f"{outdir}/{subswath}_COH_geo.tif", 'r') as src:
        prof = src.profile.copy()
        print(prof)
        prof.update({"driver": "GTiff",
                "nodata": 0
                 })
        struct = np.ones((15,15))
        with rio.open(f"{outdir}/{subswath}_COH_geo_border.tif", 'w', **prof) as dst:
            for i in range(1, prof['count'] + 1):
                band_src = src.read(i)
                msk_src = band_src != 0
                msk_dst = binary_erosion(msk_src, struct)
                band_dst = band_src * msk_dst
                dst.write(band_dst, i)

In [ ]:
ds_iw2 = rio.open(f"{outdir}/IW2_COH_geo_border.tif")
ds_iw3 = rio.open(f"{outdir}/IW3_COH_geo_border.tif")
arr_merge, trans_merge = merge.merge([ds_iw2, ds_iw3])
with rio.open(f"{outdir}/IW2_COH_geo_border.tif") as src:
        out_meta = src.meta.copy()    
out_meta.update({
                 "height": arr_merge.shape[1],
                 "width": arr_merge.shape[2],
                 "transform": trans_merge,
                 "nodata": 0
                 })

# crop without writing intermediate file
with MemoryFile() as memfile:
    with memfile.open(**out_meta) as mem:
        # Populate the input file with numpy array
        mem.write(arr_merge)
        arr_crop, trans_crop = mask.mask(mem, [aoi.geometry[0]], crop=True)
        prof_crop = mem.profile.copy() 
        prof_crop.update(
                {
                    "transform": trans_crop,
                    "width": arr_crop.shape[2],
                    "height": arr_crop.shape[1]
                }
             )

# write as COG
with MemoryFile() as memfile:
    with memfile.open(**prof_crop) as mem:
        mem.write(arr_crop)
        dst_profile = cog_profiles.get("deflate")
        cog_translate(
            mem,
            f"{outdir}/mergedRasters_masked.tif",
            dst_profile,
            in_memory=True,
            quiet=True,
        )

In [24]:
with rio.open(f"{outdir}/IW2_COH_geo.tif") as src:
        out_meta = src.profile.copy()    
        tags = src.tags(1).copy() 

In [51]:
# output file name
from datetime import datetime
import calendar

pol = 'VV'
subswath = 'IW1'
ids = identify_many(infiles)
meta_mst = ids[0].scanMetadata()
meta_slv = ids[1].scanMetadata()
slnum = meta_mst['sliceNumber']
orbnum = meta_mst['orbitNumber_rel']
if meta_slv['sliceNumber'] != slnum:
    raise ValueError('Images from two different slices')
if meta_slv['orbitNumber_rel'] != orbnum:
    raise ValueError('Images from two different orbits')
datestr_mst = meta_mst['start']
datestr_slv = meta_slv['start']
date_mst = datetime.strptime(datestr_mst, "%Y%m%dT%H%M%S")
date_slv = datetime.strptime(datestr_slv, "%Y%m%dT%H%M%S")
calendar_mst = f'{date_mst.day}{calendar.month_abbr[date_mst.month]}{date_mst.year}'
calendar_slv = f'{date_slv.day}{calendar.month_abbr[date_slv.month]}{date_slv.year}'
out_name = f"coh_{subswath}_{pol}_{calendar_mst}_{calendar_slv}_slnum_{slnum}"
# datetime('20230814T180526')
print(out_name)

coh_IW1_VV_26Aug2023_14Aug2023_slnum_9


In [35]:
tt = datetime.strptime('20230814T180526', "%Y%m%dT%H%M%S")

In [47]:
from pyroSAR.snap.auxil import Par_BandMath
Par_BandMath()

In [51]:
wfl_int = Workflow('../graph/MasterSlaveIntensity.xml')
wfl_int['BandMaths'].parameters.add_equation()

pyroSAR Node object 'BandMaths'